<a href="https://colab.research.google.com/github/teyang-lau/coffee-joint-rec-sys/blob/main/RankFM_LOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --quiet geopandas

     |████████████████████████████████| 1.0 MB 15.8 MB/s 
     |████████████████████████████████| 16.7 MB 52.9 MB/s 
     |████████████████████████████████| 6.3 MB 50.6 MB/s 


In [3]:
!pip install --quiet plotly_express

In [4]:
!pip install --quiet geopy

In [5]:
!pip install --quiet rankfm

     |████████████████████████████████| 145 kB 14.2 MB/s 


In [7]:
import requests
import numpy as np
import pandas as pd
import io

import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook

username = 'shaunaloh'
token = 'xxxx'
github_session = requests.Session()
github_session.auth = (username,token)
train_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/train_lol.csv'
val_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/val_lol.csv'
test_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/test_lol.csv'
coffee_url = 'https://raw.githubusercontent.com/teyang-lau/coffee-joint-rec-sys/main/data/processed/sg_coffee_shops_final.csv'

In [8]:
def get_data(url,username=username,token=token):
  download = github_session.get(url).content
  df = pd.read_csv(io.StringIO(download.decode('utf-8')))
  return df

In [9]:
train = get_data(train_url)
train = train[['userid','shop','rating']]
train.rename({'userid':'user_id','shop':'item_id'},axis=1,inplace=True)
train.head()

,user_id,item_id,rating
0,-5YMIME_WEin_by41Bj-3Q,liho-tea-singapore-117,3.0
1,-5YMIME_WEin_by41Bj-3Q,old-hen-coffee-bar-singapore-2,4.0
2,-5YMIME_WEin_by41Bj-3Q,two-men-bagel-house-singapore,4.0
3,-5YMIME_WEin_by41Bj-3Q,old-airport-road-food-centre-singapore,5.0
4,-G1YjYxjDpxOBzFgo36ORA,the-book-cafe-singapore,3.0


In [10]:
val = get_data(val_url)
val = val[['userid','shop','rating']]
val.rename({'userid':'user_id','shop':'item_id'},axis=1,inplace=True)
val.head()

,user_id,item_id,rating
0,-5YMIME_WEin_by41Bj-3Q,dutch-colony-coffee-singapore,4.0
1,-G1YjYxjDpxOBzFgo36ORA,the-providore-singapore,1.0
2,-WShM_YFbtG4OcE0vrFVyw,nylon-coffee-roasters-singapore,4.0
3,-XaIf12ricWc5z5BRt9nnQ,tiong-bahru-bakery-singapore-4,4.0
4,-fUWq6sOIEe1uTUhNKS9sQ,drips-singapore,4.0


In [11]:
test = get_data(test_url)
test = test[['userid','shop','rating']]
test.rename({'userid':'user_id','shop':'item_id'},axis=1,inplace=True)
test.head()

,user_id,item_id,rating
0,-5YMIME_WEin_by41Bj-3Q,luna-singapore,3.0
1,-G1YjYxjDpxOBzFgo36ORA,common-man-coffee-roasters-singapore,2.0
2,-WShM_YFbtG4OcE0vrFVyw,chye-seng-huat-hardware-singapore,3.0
3,-XaIf12ricWc5z5BRt9nnQ,chye-seng-huat-hardware-singapore,3.0
4,-fUWq6sOIEe1uTUhNKS9sQ,meng-kitchen-singapore,5.0


In [12]:
coffee = get_data(coffee_url)
coffee =coffee[['alias','price','address','latitude','longitude']]
coffee.head()

,alias,price,address,latitude,longitude
0,nylon-coffee-roasters-singapore,$,"4 Everton Park, # 01-40, Singapore 080004, Sin...",1.276810,103.839976
1,five-oars-coffee-roasters-singapore-2,$$,"39 Tanjong Pagar Rd, Singapore 088462, Singapore",1.279030,103.843920
2,the-wired-monkey-singapore-singapore,$,"5 Dunlop St, Singapore 209335, Singapore",1.304320,103.854090
3,glasshouse-singapore,$$,"30 Victoria St, #01-03, CHIJMES, Singapore 187...",1.294508,103.851137
4,tiong-hoe-specialty-coffee-singapore-3,$,"170 Stirling Rd, #01-1133, Singapore 140170, S...",1.290920,103.803080


Geo-binning

In [13]:
from copy import deepcopy
coffee_ = deepcopy(coffee)
coffee_.head()

,alias,price,address,latitude,longitude
0,nylon-coffee-roasters-singapore,$,"4 Everton Park, # 01-40, Singapore 080004, Sin...",1.276810,103.839976
1,five-oars-coffee-roasters-singapore-2,$$,"39 Tanjong Pagar Rd, Singapore 088462, Singapore",1.279030,103.843920
2,the-wired-monkey-singapore-singapore,$,"5 Dunlop St, Singapore 209335, Singapore",1.304320,103.854090
3,glasshouse-singapore,$$,"30 Victoria St, #01-03, CHIJMES, Singapore 187...",1.294508,103.851137
4,tiong-hoe-specialty-coffee-singapore-3,$,"170 Stirling Rd, #01-1133, Singapore 140170, S...",1.290920,103.803080


In [14]:
coffee_['latitude']=coffee_['latitude'].astype(str)
coffee_['longitude']=coffee_['longitude'].astype(str)

In [15]:
geolocator = Nominatim(user_agent="geoapiExercises")
region = []
for i in range(len(coffee_)):
  search = coffee_['latitude'].loc[i]+','+coffee_['longitude'].loc[i]
  location = list(geolocator.geocode(search))
  location = location[0].split(',')
  region.append(location[-3])
  if i%100 ==0:
    print(f'==Processed {i} records out of {len(coffee_)}')
print(region[:5])

==Processed 0 records out of 983
==Processed 100 records out of 983
==Processed 200 records out of 983
==Processed 300 records out of 983
==Processed 400 records out of 983
==Processed 500 records out of 983
==Processed 600 records out of 983
==Processed 700 records out of 983
==Processed 800 records out of 983
==Processed 900 records out of 983
[' Central', ' Central', ' Central', ' Central', ' Central']


In [16]:
coffee_['region']=region
coffee_.head()

,alias,price,address,latitude,longitude,region
0,nylon-coffee-roasters-singapore,$,"4 Everton Park, # 01-40, Singapore 080004, Sin...",1.276809987029,103.83997606961,Central
1,five-oars-coffee-roasters-singapore-2,$$,"39 Tanjong Pagar Rd, Singapore 088462, Singapore",1.27903,103.84392,Central
2,the-wired-monkey-singapore-singapore,$,"5 Dunlop St, Singapore 209335, Singapore",1.30432,103.85409,Central
3,glasshouse-singapore,$$,"30 Victoria St, #01-03, CHIJMES, Singapore 187...",1.29450802017737,103.85113726683,Central
4,tiong-hoe-specialty-coffee-singapore-3,$,"170 Stirling Rd, #01-1133, Singapore 140170, S...",1.29092,103.80308,Central


Developing item features matrix

In [17]:
dict(coffee_['region'].value_counts())

{' Central': 752,
 ' Northeast': 17,
 ' Northwest': 49,
 ' Southeast': 104,
 ' Southwest': 58,
 ' 益阳市': 3}

In [18]:
coffee_[coffee_['region']==' 益阳市']

,alias,price,address,latitude,longitude,region
588,charlie-brown-cafe-singapore-5,$$,"11 Tanjong Katong Rd, #01-52, OneKM Shopping M...",nan,nan,益阳市
612,cha-tra-mue-no-title,$$,"137 Amoy St, #01-05, Far East Square, Singapor...",nan,nan,益阳市
930,rochor-food-town-singapore,$$,"Singapore, Singapore",nan,nan,益阳市


In [21]:
print(coffee_[coffee_['address'].str.contains('Tanjong Katong')]['region'])
print('\n',coffee_[coffee_['address'].str.contains('Amoy St')]['region'])

78      Southeast
588           益阳市
680     Southeast
Name: region, dtype: object

 66      Central
197     Central
518     Central
560     Central
564     Central
597     Central
612         益阳市
753     Central
779     Central
Name: region, dtype: object


In [24]:
#Manual adjustments to '益阳市'
coffee_['region'].loc[596] = ' Southeast'
coffee_['region'].loc[621] = ' Central'
coffee_['region'].loc[658] = ' Southeast'
coffee_['region'].loc[947] = ' Central'
coffee_['region'].loc[588] = ' Southeast'
coffee_['region'].loc[612] = ' Central'
coffee_['region'].loc[930] = ' Central'
coffee_['region'].value_counts()

 Central      754
 Southeast    105
 Southwest     58
 Northwest     49
 Northeast     17
Name: region, dtype: int64

In [25]:
from google.colab import drive
drive.mount('/content/drive')
%cd cont
coffee_.to_csv('coffee_binned.csv')

Mounted at /content/drive
[Errno 2] No such file or directory: 'cont'
/content


In [26]:
price_ = pd.get_dummies(coffee_['price'])
region_ = pd.get_dummies(coffee_['region'])
region_.head()

,Central,Northeast,Northwest,Southeast,Southwest
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [27]:
frames = [coffee_,price_,region_]
coffee_f = pd.concat(frames,axis=1)
coffee_f=coffee_f.drop(['price','address','latitude','longitude','region'],axis=1)
coffee_f.rename(columns = {'alias':'item_id'}, inplace = True)
coffee_f.head()

,item_id,$,$$,$$$,$$$$,Central,Northeast,Northwest,Southeast,Southwest
0,nylon-coffee-roasters-singapore,1,0,0,0,1,0,0,0,0
1,five-oars-coffee-roasters-singapore-2,0,1,0,0,1,0,0,0,0
2,the-wired-monkey-singapore-singapore,1,0,0,0,1,0,0,0,0
3,glasshouse-singapore,0,1,0,0,1,0,0,0,0
4,tiong-hoe-specialty-coffee-singapore-3,1,0,0,0,1,0,0,0,0


In [33]:
interactions = pd.concat([train,val,test], axis=0)
interactions.head()

,user_id,item_id,rating
0,-5YMIME_WEin_by41Bj-3Q,liho-tea-singapore-117,3.0
1,-5YMIME_WEin_by41Bj-3Q,old-hen-coffee-bar-singapore-2,4.0
2,-5YMIME_WEin_by41Bj-3Q,two-men-bagel-house-singapore,4.0
3,-5YMIME_WEin_by41Bj-3Q,old-airport-road-food-centre-singapore,5.0
4,-G1YjYxjDpxOBzFgo36ORA,the-book-cafe-singapore,3.0


In [34]:
interactions['item_id'].nunique()

856

In [29]:
#check
items = list(coffee_f['item_id'].unique())
missing = []
for item in items:
  if item not in list(interactions['item_id'].unique()):
    missing.append(item)

missing

['glasshouse-singapore',
 'craftsmen-specialty-coffee-singapore-4',
 'hygge-singapore',
 'apartment-coffee-singapore',
 'arabica-singapore-3',
 'bettr-barista-coffee-academy-singapore',
 'tcc-the-connoisseur-concerto-singapore-13',
 'the-autobus-singapore',
 'upside-down-coffee-alternatives-singapore',
 'dutch-colony-coffee-singapore-2',
 'the-quarters-singapore',
 'o-coffee-club-singapore-4',
 'chaiholics-singapore-3',
 'ya-kun-kaya-toast-singapore-42',
 'ten-cafe-singapore',
 'tai-hwa-coffee-stall-singapore',
 'compound-coffee-co-singapore',
 'the-coffee-stop-singapore',
 'o-coffee-club-singapore-9',
 'limaa-singapore',
 'got-luck-cafe-singapore',
 'laaf-singapore',
 'brew-maison-singapore',
 'starbucks-singapore-103',
 'ya-kun-kaya-toast-singapore-48',
 'quan-chin-cafe-singapore',
 'killiney-singapore-8',
 'dot-and-dash-singapore',
 'the-coffee-bean-and-tea-leaf-singapore-38',
 'k88-coffee-house-singapore',
 'cafe-nido-singapore',
 'ya-kun-kaya-toast-singapore-40',
 'ya-kun-kaya-toa

In [41]:
# remove coffee places from coffee_f dataframe, which does not exist in interactions dataframe.
item_features = coffee_f[~coffee_f['item_id'].isin(missing)]

In [36]:
coffee_f['item_id'].nunique()

856

In [37]:
# print(len(coffee_f))
# item_features=coffee_f[~coffee_f['item_id'].isin(missing)]
# print(len(item_features))
# item_features.head()

In [38]:
from rankfm.rankfm import RankFM
from rankfm.evaluation import hit_rate, reciprocal_rank, discounted_cumulative_gain, precision, recall, diversity

In [48]:
#check matrix/vector dimensions
unique_users = interactions.user_id.nunique()
unique_items = interactions.item_id.nunique()

print("interactions shape: {}".format(interactions.shape))
print("interactions unique users: {}".format(interactions.user_id.nunique()))
print("interactions unique items: {}".format(interactions.item_id.nunique()))

print("user features users:", interactions.user_id.nunique())
print("item features items:", interactions.item_id.nunique())

interactions shape: (3816, 3)
interactions unique users: 497
interactions unique items: 856
user features users: 497
item features items: 856


In [39]:
unique_users = train['user_id'].nunique()
unique_items = train['item_id'].nunique()
sparsity = 1 - (len(train) / (unique_users * unique_items))
print("interaction matrix sparsity: {}%".format(round(100 * sparsity, 1)))

interaction matrix sparsity: 99.2%


In [40]:
train_items = np.sort(train.item_id.unique())
train_items[:5]

array(['1983-a-taste-of-nanyang-singapore-2',
       '238-coffee-stall-singapore', '2am-dessert-bar-singapore',
       '2nd-mini-steamboat-delight-singapore',
       '5-the-moments-cafe-singapore'], dtype=object)

In [42]:
item_features_train = item_features[item_features['item_id'].isin(train_items)].reset_index()
item_features_train.drop(['index'],axis=1,inplace=True)
item_features_train.head()

,item_id,$,$$,$$$,$$$$,Central,Northeast,Northwest,Southeast,Southwest
0,nylon-coffee-roasters-singapore,1,0,0,0,1,0,0,0,0
1,five-oars-coffee-roasters-singapore-2,0,1,0,0,1,0,0,0,0
2,the-wired-monkey-singapore-singapore,1,0,0,0,1,0,0,0,0
3,tiong-hoe-specialty-coffee-singapore-3,1,0,0,0,1,0,0,0,0
4,mellower-coffee-singapore-188967,0,1,0,0,1,0,0,0,0


In [43]:
#check
sorted(train['item_id'].unique())==sorted(item_features_train['item_id'].unique())

True

In [44]:
val_items = np.sort(val.item_id.unique())
item_features_val = item_features[item_features['item_id'].isin(val_items)].reset_index()
item_features_val.drop(['index'],axis=1,inplace=True)
item_features_val.head()

,item_id,$,$$,$$$,$$$$,Central,Northeast,Northwest,Southeast,Southwest
0,nylon-coffee-roasters-singapore,1,0,0,0,1,0,0,0,0
1,mellower-coffee-singapore-188967,0,1,0,0,1,0,0,0,0
2,common-man-coffee-roasters-singapore,0,1,0,0,1,0,0,0,0
3,the-coffee-academics-singapore-3,0,1,0,0,1,0,0,0,0
4,chye-seng-huat-hardware-singapore,0,1,0,0,1,0,0,0,0


In [45]:
train_set = train[['user_id','item_id']] #remove the ratings which is the target

In [51]:
val_set = val[['user_id','item_id']]
test_set = test[['user_id', 'item_id']]

Developing the Factorization Machine

In [63]:
model = RankFM(factors=50, 
               loss='warp', 
               max_samples=10, #maximum negative samples
               alpha=0.01, 
               sigma=0.1, 
               learning_rate=0.01, 
               learning_schedule='invscaling') #inverse scaling to decrease learning rate over time
model

In [64]:
model.fit(train_set, 
          item_features=item_features_train, 
          epochs=20, 
          verbose=True)


training epoch: 0
log likelihood: -1944.81005859375

training epoch: 1
log likelihood: -1894.1700439453125

training epoch: 2
log likelihood: -1855.25

training epoch: 3
log likelihood: -1821.5400390625

training epoch: 4
log likelihood: -1789.3599853515625

training epoch: 5
log likelihood: -1766.9200439453125

training epoch: 6
log likelihood: -1742.4300537109375

training epoch: 7
log likelihood: -1733.8199462890625

training epoch: 8
log likelihood: -1714.1400146484375

training epoch: 9
log likelihood: -1690.8499755859375

training epoch: 10
log likelihood: -1675.6700439453125

training epoch: 11
log likelihood: -1661.300048828125

training epoch: 12
log likelihood: -1647.949951171875

training epoch: 13
log likelihood: -1648.719970703125

training epoch: 14
log likelihood: -1637.3699951171875

training epoch: 15
log likelihood: -1627.8199462890625

training epoch: 16
log likelihood: -1623.6400146484375

training epoch: 17
log likelihood: -1610.2099609375

training epoch: 18
log 

In [65]:
valid_scores = model.predict(val_set) #, cold_start='nan') 
print(valid_scores.shape)
pd.Series(valid_scores).describe()

(497,)


count    433.000000
mean       0.300460
std        0.403206
min       -0.379245
25%       -0.032474
50%        0.233068
75%        0.554645
max        1.465840
dtype: float64

Generate Top 20 Recommendations for Validation Users

In [68]:
valid_recommendations = model.recommend(val_set.user_id, n_items=20, filter_previous=True) #, cold_start='nan')
valid_recommendations.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
user_id,,,,,,,,,,,,,,,,,,,,
-5YMIME_WEin_by41Bj-3Q,nylon-coffee-roasters-singapore,maxwell-food-centre-singapore-3,chye-seng-huat-hardware-singapore,group-therapy-coffee-singapore,tiong-bahru-bakery-singapore-4,toms-palette-singapore,ps-cafe-singapore-8,lau-pa-sat-singapore-2,oriole-coffee-bar-singapore-2,the-book-cafe-singapore,wild-honey-singapore-3,sunday-folks-singapore,tiong-bahru-food-market-and-hawker-centre-sing...,food-for-thought-singapore,strangers-reunion-singapore,common-man-coffee-roasters-singapore,pacamara-boutique-coffee-roasters-singapore,d-good-cafe-singapore,violet-oons-kitchen-singapore,kith-cafe-singapore
-G1YjYxjDpxOBzFgo36ORA,maxwell-food-centre-singapore-3,nylon-coffee-roasters-singapore,group-therapy-coffee-singapore,tiong-bahru-bakery-singapore-4,chye-seng-huat-hardware-singapore,lau-pa-sat-singapore-2,oriole-coffee-bar-singapore-2,toms-palette-singapore,ps-cafe-singapore-8,sunday-folks-singapore,old-airport-road-food-centre-singapore,wild-honey-singapore-3,tiong-bahru-food-market-and-hawker-centre-sing...,two-men-bagel-house-singapore,strangers-reunion-singapore,common-man-coffee-roasters-singapore,pacamara-boutique-coffee-roasters-singapore,food-for-thought-singapore,violet-oons-kitchen-singapore,the-assembly-ground-singapore
-WShM_YFbtG4OcE0vrFVyw,nylon-coffee-roasters-singapore,maxwell-food-centre-singapore-3,chye-seng-huat-hardware-singapore,tiong-bahru-bakery-singapore-4,toms-palette-singapore,ps-cafe-singapore-8,lau-pa-sat-singapore-2,old-airport-road-food-centre-singapore,wild-honey-singapore-3,oriole-coffee-bar-singapore-2,common-man-coffee-roasters-singapore,sunday-folks-singapore,tiong-bahru-food-market-and-hawker-centre-sing...,food-for-thought-singapore,two-men-bagel-house-singapore,the-book-cafe-singapore,tobys-estate-singapore,nassim-hill-bakery-bistro-bar-singapore,kith-cafe-singapore,ps-cafe-singapore-7
-XaIf12ricWc5z5BRt9nnQ,maxwell-food-centre-singapore-3,nylon-coffee-roasters-singapore,group-therapy-coffee-singapore,chye-seng-huat-hardware-singapore,sunday-folks-singapore,toms-palette-singapore,tiong-bahru-bakery-singapore-4,lau-pa-sat-singapore-2,ps-cafe-singapore-8,wild-honey-singapore-3,old-airport-road-food-centre-singapore,two-men-bagel-house-singapore,tiong-bahru-food-market-and-hawker-centre-sing...,paul-singapore-3,oriole-coffee-bar-singapore-2,the-book-cafe-singapore,food-for-thought-singapore,strangers-reunion-singapore,common-man-coffee-roasters-singapore,sarnies-singapore
-fUWq6sOIEe1uTUhNKS9sQ,maxwell-food-centre-singapore-3,nylon-coffee-roasters-singapore,group-therapy-coffee-singapore,chye-seng-huat-hardware-singapore,lau-pa-sat-singapore-2,tiong-bahru-bakery-singapore-4,old-airport-road-food-centre-singapore,two-men-bagel-house-singapore,the-book-cafe-singapore,strangers-reunion-singapore,toms-palette-singapore,wild-honey-singapore-3,ps-cafe-singapore-8,oriole-coffee-bar-singapore-2,paul-singapore-3,kith-cafe-singapore,tiong-bahru-food-market-and-hawker-centre-sing...,food-for-thought-singapore,level33-singapore-4,sunday-folks-singapore


In [69]:
#save recommendations to csv
valid_recommendations.to_csv("RankFM_LOL_recommendations.csv", index=False)
from google.colab import files
files.download("RankFM_LOL_recommendations.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model Evaluation

In [67]:
%%time
k=5
model_hit_rate = hit_rate(model, val_set, k=k)
model_reciprocal_rank = reciprocal_rank(model, val_set, k=k)
model_dcg = discounted_cumulative_gain(model, val_set, k=k)
model_precision = precision(model, val_set, k=k)
model_recall = recall(model, val_set, k=k)

print("hit_rate: {:.3f}".format(model_hit_rate))
print("reciprocal_rank: {:.3f}".format(model_reciprocal_rank))
print("dcg: {:.3f}".format(model_dcg, 3))
print("precision: {:.3f}".format(model_precision))
print("recall: {:.3f}".format(model_recall))

hit_rate: 0.046
reciprocal_rank: 0.035
dcg: 0.038
precision: 0.009
recall: 0.046
CPU times: user 1.16 s, sys: 0 ns, total: 1.16 s
Wall time: 1.16 s
